<a href="https://colab.research.google.com/github/nunocesarsa/Examples/blob/main/GEE_Clipper_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing Earth Engine package
import ee

#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=GJW4BdFklQxameDTP_Yn3GR3iyptIZ--_qv1VilLVsw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5f5ZiA_p2nWWtEZw0chS6o7nYY-B9A3Dix2dfE2zTQSkJYz75cd2c

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyproj

     |████████████████████████████████| 6.6MB 5.1MB/s 


In [ ]:
from pyproj import Proj, transform

In [ ]:
!pip uninstall geopy
!pip install geopy

     |████████████████████████████████| 112kB 5.0MB/s 


In [ ]:
import geopy
geopy.__version__

#requires version 2.10 at least - notice colab uses 1.17.0 (06062021)

'2.1.0'

In [ ]:
#packages required
from pyproj import Proj, Transformer


#Possibitlities:
# Generate an array with the different corners
def extent_creator(x_in,y_in,CRS_in='epsg:4326',side_length=50):

  #i could generate 
  if CRS_in != 'EPSG:4326':

    #outval = transform(CRS_in,'epsg:4326',x_in,y_in)
    outval = Transformer.from_crs(CRS_in, 'epsg:4326').transform(x_in,y_in)

  #outval = x_in,y_in
  return(outval)


#task_creator(10,20)
#using coordinates from here to test: https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python
extent_creator(11705274.6374,4826473.6922,'epsg:3857')

(39.72785727727917, 105.15027111593008)

In [ ]:
def square_creator(x_in,y_in,side_len=10):
  
  side_half = side_len/2

  x_max = x_in + side_half
  x_min = x_in - side_half

  y_max = y_in + side_half
  y_min = y_in - side_half

  outval = x_max, x_min, y_max, y_min

  return outval

square_creator(-25,-10,5)

(-22.5, -27.5, -7.5, -12.5)

In [ ]:
#import numpy as np
from pyproj import Proj, Transformer
import geopy
from geopy.distance import geodesic


class GEE_Clipper:
  def __init__(self, x=0,y=0,CRS='epsg:4326',side_len=10):
    self.x = x
    self.y = y
    self.CRS =CRS
    self.side_len = side_len 

  #just returns a dictionary of the input data
  def get_data(self):
    return {'x':self.x,
            'y':self.y,
            'CRS':self.CRS,
            'Side length':self.side_len}

  #in case the data is already projected
  #outputs an "extent object"
  def square_creator_projected(self,poly=False):
    side_half = self.side_len/2

    x_max = self.x + side_half
    x_min = self.x - side_half

    y_max = self.y + side_half
    y_min = self.y - side_half

    if poly == False:
     return [x_max, x_min, y_max, y_min]
    else:
     return [[x_max, y_max],[x_max, y_min],[x_min, y_min],[x_min, y_max]]
    
  #in case the data is not projected
  #outputs n extent object
  #adapted from here https://stackoverflow.com/questions/24427828/calculate-point-based-on-distance-and-direction - many of the documentation on the original package is a bit wacky
  def square_creator_geographic(self,poly=False):
    
    x_deg, y_deg =  Transformer.from_crs(self.CRS, 'epsg:4326').transform(self.x,self.y)
    side_half = geopy.distance.GeodesicDistance(meters=self.side_len/2)

    lat_nort = side_half.destination(point=(y_deg,x_deg),bearing=0).latitude
    lat_sout = side_half.destination(point=(y_deg,x_deg),bearing=180).latitude
    lon_east = side_half.destination(point=(y_deg,x_deg),bearing=90).longitude
    lon_west = side_half.destination(point=(y_deg,x_deg),bearing=270).longitude

    if poly == False:
      return [lon_east, lon_west,lat_nort, lat_sout]
    else:
      return [[lat_nort,lon_east],[lat_sout,lon_east],[lat_sout,lon_west],[lat_nort,lon_west]]
     #return [[lon_east,lat_nort],[lon_east,lat_sout],[lon_west,lat_sout],[lon_west,lat_nort]]

  #simply organizes the output of the square creator to a polygon that can be read into a GEE object
  #def generate_polygon(ext_obj):
  #  TR = [ext_obj[0],ext_obj[2]]
  #  BR = [ext_obj[0],ext_obj[3]]
  #  BL = [ext_obj[1],ext_obj[3]]
  #  TL = [ext_obj[1],ext_obj[2]]

  #  return [[TR],[BR],[BL],[TL]]

  #requires pyproj
  #def convertToWGS84(self.x,self.y,self.side_len):
  def convertToWGS84(self):

    if self.CRS == 'epsg:4326':
      return [self.x,self.y,self.CRS,self.side_len]
    else:
      x_deg, y_deg =  Transformer.from_crs(self.CRS, 'epsg:4326').transform(self.x,self.y)
      
      return [x_deg,y_deg,'epsg:4326',self.side_len]


#test_obj = GEE_Clipper.square_creator(25,25)
#ee.Geometry.Polygon(GEE_Clipper.generate_polygon(test_obj))

#GEE_Clipper.get_data()
GEE_Clipper(11705274.6374,4826473.6922,'epsg:3857').get_data()

{'CRS': 'epsg:3857', 'Side length': 10, 'x': 11705274.6374, 'y': 4826473.6922}

## Test area

In [ ]:
11705274.6374-10000


11695274.6374

In [ ]:
test_obj = GEE_Clipper(-105.15027111593008,39.72785727727917 ,side_len=10000)
test_obj.get_data()
bb=test_obj.square_creator_geographic(poly=True)
bb


[[39.77289021642765, -105.09194976806823],
 [39.68282398775344, -105.09194976806823],
 [39.68282398775344, -105.20859246379193],
 [39.77289021642765, -105.20859246379193]]

In [ ]:
import geopy.distance
d = geopy.distance.distance(meters=100000)

d.destination(point=(39, 105), bearing=90)

Point(38.99428969644511, 106.15432059225918, 0.0)

In [ ]:
a= d.destination(point=(39.72785727727917, 105.15027111593008), bearing=180)
a.latitude

38.82712505238111

In [ ]:
GEE_Clipper(11705274.6374,4826473.6922,'epsg:3857').square_creator_geographic()
#GEE_Clipper(11705274.6374,4826473.6922,'epsg:4326').convertToWGS84()

(39.72785727727917, 105.15027111593008)

In [ ]:
geopy.distance.VincentyDistance(kilometers = 1)

Distance(1.0)

In [ ]:
# Define starting point.
start = geopy.Point(48.853, 2.349)

# Define a general distance object, initialized with a distance of 1 km.
d = geopy.distance.great_circle(kilometers=1)

# Use the `destination` method with a bearing of 0 degrees (which is north)
# in order to go from point `start` 1 km to north.
import math
final = d.destination(point=start, bearing=0)
final

Point(48.86199320335494, 2.349, 0.0)

In [ ]:
d.destination(point=start, bearing=0)

In [ ]:
geopy.__version__

'1.17.0'

In [ ]:
aa= GEE_Clipper(-105.15027111593008,39.72785727727917 ,side_len=10000)
ee.Geometry.Polygon(aa.square_creator_geographic(poly=True))

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              39.77289021642765,
              -105.09194976806823
            ],
            [
              39.68282398775344,
              -105.09194976806823
            ],
            [
              39.68282398775344,
              -105.20859246379193
            ],
            [
              39.77289021642765,
              -105.20859246379193
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [ ]:
poly_area_coord = [[[4.2679539239374575,52.097753553999],
                    [4.4272556817499575,52.097753553999],
                    [4.4272556817499575,52.21149891284465],
                    [4.2679539239374575,52.21149891284465],
                    [4.2679539239374575,52.097753553999]]]

ee.Geometry.Polygon(poly_area_coord)              

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              4.2679539239374575,
              52.097753553999
            ],
            [
              4.4272556817499575,
              52.097753553999
            ],
            [
              4.4272556817499575,
              52.21149891284465
            ],
            [
              4.2679539239374575,
              52.21149891284465
            ],
            [
              4.2679539239374575,
              52.097753553999
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [ ]:
type(GEE_Clipper.square_creator(-25,-10,5))

list

In [ ]:
GEE_Clipper

2.5

In [ ]:
from pyproj import Transformer

transformer = Transformer.from_crs('epsg:3857', 'epsg:4326')
transformer.transform(11705274.6374,4826473.6922)

(39.72785727727917, 105.15027111593008)

In [ ]:
Transformer.from_crs('epsg:3857', 'epsg:4326').transform(11705274.6374,4826473.6922)

(39.72785727727917, 105.15027111593008)

In [ ]:
class ComplexNumber:
    def __init__(self, r=0, i=0):
        self.real = r
        self.imag = i

    def get_data(self):
        print(f'{self.real}+{self.imag}j')

In [ ]:
# Create a new ComplexNumber object
num1 = ComplexNumber(2, 3)
num1.get_data()

2+3j
